# 智能体开发流程

![img_1.png](./images/img_1.png)

# Python环境搭建

## ANACONDA

python官网：https://www.python.org/

conda：https://github.com/conda/conda

anaconda：https://www.anaconda.com/（下载地址：https://www.anaconda.com/download）
conda、miniconda、anaconda的区别：https://juejin.cn/post/7262280335978987557

## UV

python包管理工具uv：https://github.com/astral-sh/uv

installation | uv：https://docs.astral.sh/uv/getting-started/installation/

阿里云源地址：https://mirrors.aliyun.com/pypi/simple/

清华源地址：https://pypi.tuna.tsinghua.edu.cn/simple

uv配置阿里云源：

在 pyproject.toml 中添加：
```Bash

[[tool.uv.index]]
url = "https://mirrors.aliyun.com/pypi/simple/"
default = true
```

或在 uv.toml 中添加：
```Bash
[[index]]
url = "https://mirrors.aliyun.com/pypi/simple/"
default = true
```

pip配置阿里云源：

方法1：直接使用
```Bash
pip install some-package -i https://mirrors.aliyun.com/pypi/simple/
```

方法2：配置使用

找到或创建pip的配置文件。在Unix或MacOS上，通常位于`~/.pip/pip.conf`（或`~/.config/pip/pip.conf`，取决于pip版本和操作系统）。在Windows上，通常位于`%HOMEPATH%\pip\pip.ini`。

在配置文件中添加或修改以下内容：
```Plain Text
[global]
index-url = https://mirrors.aliyun.com/pypi/simple/
```

保存文件并关闭。下次使用pip时，它将默认使用阿里云的镜像。

## PyCharm

官网下载地址：https://www.jetbrains.com.cn/pycharm/download

## 基础教程

Python | 菜鸟教程：https://www.runoob.com/python/python-install.html



# 大模型调用方法

## 方法1：Ollama 本地调用

### 部署Ollama

官网：https://ollama.com/

Ollama 是一个开源的本地化大语言模型（LLM）运行和管理工具，旨在简化大型语言模型的部署和使用。它支持在本地设备上高效运行多种开源模型（如 Llama、DeepSeek、Qwen 等），无需依赖云端服务，保障数据隐私。

### 调用Ollama大模型

使用langchain-ollama库：https://docs.langchain.com/oss/python/integrations/chat/ollama#installation

## 方法2：阿里云百炼API调用

https://bailian.console.aliyun.com/

### 注意：qwen3的调用方法中增加了enable_thinking参数

`qwen3` 系列大模型在调用时，需要传入 `enable_thinking` 参数，用于控制是否开启思考，注意只要调用 `qwen3` 大模型，就必须传入这个参数，比如使用非流式输出时，调用代码：

```python
completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen3-235b-a22b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"},
    ],
    # Qwen3模型通过enable_thinking参数控制思考过程（开源版默认True，商业版默认False）
    # 使用Qwen3开源版模型时，若未启用流式输出，请将下行取消注释，否则会报错
    extra_body={"enable_thinking": True},
)
print(completion.model_dump_json())
```

如果使用流式输出，也必须传入，同时需要使 `stream` 属性为 `True`：
```python
completion = client.chat.completions.create(
    model="qwen3-235b-a22b",  # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': '你是谁？'}],
    stream=True,
    stream_options={"include_usage": True},
    extra_body={"enable_thinking": True},
    )
for chunk in completion:
    print(chunk.model_dump_json())
```

总结一下：
- 使用 `qwen3` 大模型必须传入 `extra_body={"enable_thinking": False}` 参数，否则运行时会报错
    - 如果非流式，则 `enable_thinking` 传入 `False`
    - 如果使用流式，则 `enable_thinking` 传入 `True`，并且保证 `stream` 参数为 `True`，否则会报错
- 使用非 `qwen3` 大模型不需要传入 `extra_body={"enable_thinking": False}` 参数，不会报错



# LangChain框架介绍

## LangChain简介
### 官网地址

python版本官网：https://python.langchain.com/

nodejs版本官网：https://js.langchain.com/

### 源码地址

python版本：https://github.com/langchain-ai/langchain

nodejs版本：https://github.com/langchain-ai/langchainjs

### LangChain简介和特性
- LangChain是一个开源的标准化框架，旨在简化基于大型语言模型（LLM）的应用程序开发流程
- 通过提供模块化的组件和工具，将LLM与其他数据源、工具和计算资源无缝连接，使开发者能够更高效地构建复杂AI应用
- 自2022年10月首次发布以来，LangChain已迅速成为GitHub上增长最快的开源项目之一
- 2024年1月，LangChain发布首个稳定版本0.1.0，目前已更新至0.3版本

### 发展现状
LangChain目前已成为一个成熟且活跃的开源项目，拥有强大的社区支持和丰富的功能扩展。截至2025年5月，LangChain的Python主仓库GitHub Star数已超过10万，成为GitHub上增长最快的AI开源项目之一。同时，其Java版本LangChain4j也于2025年初开始活跃，3月更新了多个仓库（如JeecgBoot、LangChain4j-aideepin等），5月发布了1.0-Beta3版本，接近正式版本。

### 项目架构
- 核心层：langchain-core包含主要的抽象概念、接口和核心功能，这部分代码非常稳定
- 应用层：langchain包提供通用代码库，适用于不同接口实现
- 社区层：langchain-community包含大量由社区维护的轻量级集成
- 合作伙伴层：与Hugging Face、Azure、Ollama、阿里云等企业合作推出专用集成包

### 核心功能
LangChain的核心功能模块包括：
- **模型（Models）**：提供统一接口调用各种LLM，如OpenAI的GPT系列、Anthropic的Claude系列、Google的Gemini系列，以及Hugging Face的开源模型。
- **提示（Prompts）**：优化模型输入，提升生成结果的质量，包括PromptTemplate、ChatPromptTemplate和FewShotPromptTemplate等。
- **链（Chains）**：封装多个组件的调用序列，创建复杂的工作流程，如SimpleSequentialChain等。
- **代理（Agents）**：允许模型自主调用外部工具和组件，实现多步骤任务处理，如AutoGPT和BabyAGI。
- **记忆（Memory）**：存储和检索对话数据，支持上下文感知的应用，如多轮对话系统。
- **索引（Indexes）**：组织和检索文档数据，支持RAG（检索增强生成）等应用场景。
